In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 38.2 MB/s 
     |████████████████████████████████| 120 kB 54.2 MB/s 


# **BERT**

Il modello BERT è un trasformatore bidirezionale preaddestrato su un ampio set di dati in lingua inglese su testi grezzi, senza cioè che i dati venissero etichettati in alcun modo con un processo automatico per generare input ed etichette da quei testi. Più precisamente, è stato preaddestrato con due obiettivi:

*   "modelazione del linguaggio mascherato" (**Masked Language Modeling MLM**): prendendo una frase, il modello maschera casualmente il 15% delle parole nell'input, quindi legge l'intera frase mascherata attraverso il modello e deve prevedere le parole mascherate (**Token [MASK]**). Consente al modello di apprendere una rappresentazione bidirezionale della frase.
*   "previsione della frase sucessiva" (**Next Sentence Prediction (NSP)**): il modello concatena due frasi mascherate come input durante il l'addestramento. A volte corrispondono a frasi che erano una accanto all'altra nel testo originale, a volte no. Il modello deve quindi prevedere se le due frasi si susseguivano o meno.

In questo modo, il modello apprende una rappresentazione interna della lingua inglese che può quindi essere utilizzata per estrarre funzionalità utili per le attività a valle

Paper: https://arxiv.org/pdf/1810.04805.pdf


#### Usi previsti e limitazioni
Pensato principalmente per essere perfezionato su una determinata attività che utilizza l'intera frase (potenzialmente anche mascherata) come ad esempio:

*   Sequence Classification
*   Question Answering

Non è ottimizzato per la traduzione e generazione del testo
Per l'utilizzo di BERT per la traduzione si rimanda al paper
https://aclanthology.org/D19-5611.pdf






E' possibile utilizzare questo modello direttamente tramite il package **transformers** utilizzando la funzione **pipeline**)

In [ ]:
from transformers import pipeline

In [ ]:
unmasker = pipeline('fill-mask', model='bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
unmasker("Hello! My [MASK] is Daniele.")

[{'score': 0.9982873797416687,
  'token': 1271,
  'token_str': 'name',
  'sequence': 'Hello! My name is Daniele.'},
 {'score': 0.0005912640481255949,
  'token': 10208,
  'token_str': 'Name',
  'sequence': 'Hello! My Name is Daniele.'},
 {'score': 0.0002137003612006083,
  'token': 4134,
  'token_str': 'address',
  'sequence': 'Hello! My address is Daniele.'},
 {'score': 0.00011752943100873381,
  'token': 12239,
  'token_str': 'surname',
  'sequence': 'Hello! My surname is Daniele.'},
 {'score': 0.00010872080019908026,
  'token': 2666,
  'token_str': 'names',
  'sequence': 'Hello! My names is Daniele.'}]

In [ ]:
classification_task = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
result = classification_task("I love ypu.")
result

[{'label': 'POSITIVE', 'score': 0.9998257756233215}]

In [ ]:
result = classification_task("I hate you")
result

[{'label': 'NEGATIVE', 'score': 0.9991129040718079}]

## **TOKEN**

Per utilizzare un modello BERT pre-addestrato, dobbiamo convertire i dati di input in un formato appropriato in modo che ogni frase possa essere inviata al modello

### **1. Token [CLS]**

Per l'attività di classificazione è necessario inviare un singolo vettore che rappresenti l'intera frase. E' necessario, quindi, aggiungere manualmente un tokek aggiuntivo alla frase di input **[CLS]**



In [ ]:
from transformers import BertTokenizer

In [ ]:
tz = BertTokenizer.from_pretrained("bert-base-cased")

token_cls = "[CLS]"
sent = "Let's learn deep learning!"

sentence = token_cls + sent
tz.tokenize(sentence)

['[CLS]', 'Let', "'", 's', 'learn', 'deep', 'learning', '!']

Dopo aver suddiviso la frase in **token** la devo convertire in un formato appropriato per il modello **ID**

In [ ]:
tz.convert_tokens_to_ids(tz.tokenize(sentence))

[101, 2421, 112, 188, 3858, 1996, 3776, 106]

### **2. Token [SEP]**

Per l'attività di "previsione della frase sucessiva" si ha bisogno di informare il modello dove finisce la prima frase (**Sentence A**) e dove inizia la seconda frase (**Sentence B**).
Viene, quindi, introdotto un nuovo token **[SEP]**. 

N.B. Nei task di classificazione ogni frase di input conterrà il token **[SEP]** che verrà aggiunto alla fine della frase.


In [ ]:
token_sep = "[SEP]"

sentence = token_cls + sent + token_sep
tz.tokenize(sentence)

['[CLS]', 'Let', "'", 's', 'learn', 'deep', 'learning', '!', '[SEP]']

In [ ]:
ids = tz.convert_tokens_to_ids(tz.tokenize(sentence))
ids

[101, 2421, 112, 188, 3858, 1996, 3776, 106, 102]

### **3. Token [PAD]**

Il modello BERT riceve come input una lunghezza fissa della frase. La lunghezza massima di una frase dipende dai dati su cui stiamo lavorando. Per le frasi che sono più brevi di questa lunghezza massima, dovremo aggiungere padding (token vuoti) alle frasi per comporre la lunghezza. Viene, quindi, aggiunto il token **[PAD]** (in coda alla frase) in modo da avere le frasi tutte della stessa lunghezza. Nel Modello originario è stato impostato a 512, per il nostro esempio imposteremo la lunghezza a 64

In [ ]:
max_length = 10

for token_pad in range(len(tz.tokenize(sentence)), max_length):
  sentence = sentence + "[PAD]"

sentence

"[CLS]Let's learn deep learning![SEP][PAD]"

In [ ]:
ids = tz.convert_tokens_to_ids(tz.tokenize(sentence))
ids

[101, 2421, 112, 188, 3858, 1996, 3776, 106, 102, 0]

### **4. Token [UNK]**

Quando il modello BERT è stato addestrato, a ogni token è stato assegnato un ID univoco . Quindi, quando vogliamo utilizzare un modello BERT pre-addestrato, dovremo prima convertire ogni token nella frase di input nei suoi ID univoci corrispondenti.

C'è un punto importante da notare quando utilizziamo un modello pre-addestrato.Il vocabolario utilizzato per l'addestramento del modello BERT è di 30.000 token. In altre parole, quando applichiamo un modello pre-addestrato ad altri dati, è possibile che alcuni token nei nuovi dati potrebbero non apparire nel vocabolario originario del modello pre-addestrato. Questo è comunemente noto come **out-of-vocabulary (OOV)**.

Per i token che non compaiono nel vocabolario originale, il sistema è stato progettato per sostituire le parole mancanti con un token speciale [UNK], che sta per token sconosciuto.

Tuttavia, la conversione potrebbe far perdere numerose informazioni d ai dati di inpunt. Quindi, BERT utilizza un algoritmo **WordPiece** che suddivide una parola in più sottoparole, in modo tale che anche le sottoparole comunemente viste possano essere rappresentate dal modello.

Ad esempio, la parola **characteristically** non compare nel vocabolario originale. 

In [ ]:
tz.convert_tokens_to_ids(["characteristically"]) # TOKEN [UKN] == 100

[100]

Se però utilizzo la funzione **tokenize** la parola verrà automaticamente suddivisa in 2 sottoparole 

*   **characteristic**: prefisso
*   **##ally**: suffisso, il doppio hash **##** indica che è suffisso di altre parole



In [ ]:
tz.tokenize("characteristically")

['characteristic', '##ally']

Questo aiuta a non perdere delle informazioni da passare al modello

In [ ]:
sent = "He remains characteristically confident and optimistic."
tz.tokenize(sent)

['He',
 'remains',
 'characteristic',
 '##ally',
 'confident',
 'and',
 'optimistic',
 '.']

In [ ]:
tz.convert_tokens_to_ids(tz.tokenize(sent))

[1124, 2606, 7987, 2716, 9588, 1105, 24876, 119]

### Riepilogo

Per un'attività di classificazione la frase di input dovrà eseguire i seguenti passaggi prima di essere inserita nel modello BERT:


1.   **Tokenization** suddivisione della frase in token
2.   **Adding [CLS]** aggiunta del token di inizio frase
3.   **Adding [SEP]** aggiunta del token di suddivisione/chiusura della frase 
4.   **Adding [PAD]** aggiunta del token in modo che le frasi siano della stessa lunghezza
5.   **Convert ID** conversione di ogni token negli ID corrispondenti


Example:

**Original Sentence**

Let's learn deep learning!

**Tokenized Sentence**

['Let', "'", 's', 'learn', 'deep', 'learning', '!']

**Adding [CLS] and [SEP] Tokens**

['[CLS]', 'Let', "'", 's', 'learn', 'deep', 'learning', '!', '[SEP]']

**Padding**

['[CLS]', 'Let', "'", 's', 'learn', 'deep', 'learning', '!', '[SEP]', '[PAD]']

**Converting to IDs**

[101, 2421, 112, 188, 3858, 1996, 3776, 106, 102, 0]






### Tokenization --> Transformers Package

Possiamo utilizzare le funzioni fornite dal package **transformers** per aiutarci ad eseguire tutti questi passaggi tramite la funzione **encode_plus**

In [ ]:
encoded = tz.encode_plus(
    text=sent,  # the sentence to be encoded
    add_special_tokens=True,  # Add [CLS] and [SEP]
    max_length = 64,  # maximum length of a sentence
    padding='max_length',  # Add [PAD]s
    return_attention_mask = True,  # Generate the attention mask
    return_tensors = 'tf',  # ask the function to return Tensorflow tensors
)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Get the input IDs in tensor format
input_ids = encoded['input_ids']
input_ids

<tf.Tensor: shape=(1, 64), dtype=int32, numpy=
array([[ 101, 2421,  112,  188, 3858, 1996, 3776,  106,  102,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)>

In [ ]:
# Get the attention mask in tensor format
attn_mask = encoded['attention_mask']
attn_mask

<tf.Tensor: shape=(1, 64), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>